# Data Cleaning

In [ ]:
from bs4 import BeautifulSoup
import re
import requests
import csv
import pandas as pd
import os
import glob

In [ ]:
folder_path = 'C:/Users/jack/Documents/ProductData'

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Get the most recent CSV file
most_recent_file = max(csv_files, key=os.path.getctime)

# Read the data from the most recent CSV file
df = pd.read_csv(most_recent_file)

# Lowercase the column names
df.columns = df.columns.str.lower()

# Lowercase all entries
df = df.applymap(lambda s:s.lower() if type(s) == str else s)

display(df)

In [ ]:
# Merge all csv files into one
os.chdir("C:/Users/jack/Documents/SoldDates")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

#export to csv
combined_csv.to_csv("C:/Users/jack/Documents/sold_dates_combined.csv", index=False, encoding='utf-8-sig')

# Read the data from the CSV file
df = pd.read_csv('C:/Users/jack/Documents/sold_dates_combined.csv')

# Remove empty rows
df = df.dropna(how='all')

# Display the updated DataFrame
print(df)

In [ ]:

# Align and add combined csv file to the main csv file, by Item Number


# Read the data from the most recent CSV file
# Set the path to the folder containing the CSV files
folder_path = 'C:/Users/jack/Documents/ProductData'

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Get the most recent CSV file
most_recent_file = 'C:/Users/jack/Documents/ProductData/dell+inspiron+3793+laptop+computer_output.csv'
#max(csv_files, key=os.path.getctime)


# Read the data from the most recent CSV file
df = pd.read_csv(most_recent_file)

# Lowercase the column names
df.columns = df.columns.str.lower()

# Lowercase all entries
df = df.applymap(lambda s:s.lower() if type(s) == str else s)

df2 = pd.read_csv('C:/Users/jack/Documents/sold_dates_combined.csv')

#  Rename item_number column to Item Number
df2.rename(columns={'item_number': 'item number'}, inplace=True)


# Merge the dataframes on the 'item number' column
df3 = pd.merge(df, df2, on='item number', how='left')



#df3 = df.merge(df2, on='item number', how='left')
display(df3)

# Write the DataFrame to a CSV file and display it
df3.to_csv(f'C:/Users/jack/Documents/AlignedComplete.csv', index=False)

# Check for duplicate rows in the merged dataframe
duplicates = df3[df3.duplicated()]

# Print the number of duplicate rows
print(f'The merged dataframe contains {len(duplicates)} duplicate rows.')

# Remove duplicate rows from the merged dataframe
df3.drop_duplicates(inplace=True)

# Print the number of rows in the cleaned dataframe
print(f'The cleaned dataframe contains {len(df3)} rows.')

# Check for missing values in the merged dataframe
missing_values = df3.isnull().sum()

# Print the number of missing values for each column
print(missing_values)

# Write the DataFrame to a CSV file and display it
df3.to_csv(f'C:/Users/jack/Documents/AlignedComplete.csv', index=False)

display(df3)

## Remove rows which do not contain a price

In [ ]:
# Read the data from the CSV file
df = pd.read_csv('C:/Users/jack/Documents/AlignedComplete.csv')

# Drop duplicate rows
df.drop_duplicates(inplace=True)

# Drop rows with missing 'Price' values
df.dropna(subset=['price'], inplace=True)

display(df)

## Remove job lot listings

In [ ]:
# Drop rows with 'Job lot' in the 'Title', 'Seller notes', or 'Condition' column
df = df[~df['title'].str.contains('job lot')]
df['seller notes'] = df['seller notes'].fillna('')
df = df[~df['seller notes'].str.contains('job lot')]
df['condition'] = df['condition'].astype(str) # Convert to string
df = df[~df['condition'].str.contains('job lot')]

# Drop rows with 'faulty' in the 'Title', 'Seller notes', or 'Condition' column
df = df[~df['title'].str.contains('faulty')]
df = df[~df['seller notes'].str.contains('faulty')]
df['condition'] = df['condition'].astype(str) # Convert to string
df = df[~df['condition'].str.contains('faulty')]

# Drop rows with 'spares' in the 'Title', 'Seller notes', or 'Condition' column
df = df[~df['title'].str.contains('spares')]
df = df[~df['seller notes'].str.contains('spares')]
df['condition'] = df['condition'].astype(str) # Convert to string
df = df[~df['condition'].str.contains('spares')]

# Drop rows with 'any' or 'various' or 'depends on stock' in the 'model' column
df['model'] = df['model'].fillna('')
df = df[~df['model'].str.contains('any')]
df = df[~df['model'].str.contains('various')]
df = df[~df['model'].str.contains('depends on stock')]

# Drop rows with '/' in the 'brand' or 'model' column
df['brand'] = df['brand'].fillna('')
df = df[~df['brand'].str.contains('/')]
df = df[~df['model'].str.contains('/')]

display(df)

## Remove none working devices

In [ ]:
# Drop all entries which don't contain 'Good', 'Used', 'New', 'Excellent' or 'Refurbished' in the Condition column
df = df[df['condition'].str.contains('good|used|new|excellent|refurbished')]
display(df)

## CPU generation splitter

In [ ]:
# Split Processor column into Processor i series and Processor generation columns
df['processor i series'] = df['processor'].str.extract(r'(i\d+)')
df['processor generation'] = df['processor'].str.extract(r'(\d+st|\d+nd|\d+rd|\d+th|\d+st gen|\d+nd gen|\d+rd gen|\d+th gen)')

# Reorder the columns
cols = ['price', 'brand', 'processor i series', 'processor generation',
        'processor speed', 'ram size', 'ssd capacity', 'storage type', 'screen size', 'graphics processing type', 'gpu', 'operating system', 'type', 
         'model','series', 'condition', 'processor', 'features', 'seller notes', 'title', 'link','sold_date', 'item number']
df = df[cols]
df = df.rename(columns={'ssd capacity': 'storage capacity'})

# Drop rows where 'processor i series' or 'processor generation' contain NaN entries#
df.dropna(subset=['processor i series', 'processor generation'], inplace=True)

# Strip th from 'processor generation' column
df['processor generation'] = df['processor generation'].str.replace(r'(st|rd|nd|th)', '')

display(df)

## Remove 'GHz' suffix from 'Processor Speed'

In [ ]:
# Remove 'GHz' suffix from CPU speed
df['processor speed'] = df['processor speed'].str.replace('ghz', '')

# Drop rows with NaN values in the 'processor speed' column
df.dropna(subset=['processor speed'], inplace=True)

# Drop rows with any non-float values in the 'processor speed' column
df = df[pd.to_numeric(df['processor speed'], errors='coerce').astype(float).notnull()]

# Drop rows where 'processor speed' is more than 10
df = df[df['processor speed'].astype(float) < 10]


display(df)

## Drop mutiple drive devices, for ease of beta test

In [ ]:
# Drop rows with missing storage type values
df.dropna(subset=['storage type'], inplace=True)


# Drop entries which contain 'hdd + ssd' in the 'storage type' column
df = df[~df['storage type'].str.contains('\+')]
df = df[~df['storage type'].str.contains('or')]
df = df[~df['storage type'].str.contains('and')]

# List unique entries in the 'storage type' column
print(df['storage type'].unique())


# Map storage types to categories
storage_map = {'emmc': 'emmc', 'm.2 ssd': 'm.2', 'm.2 drive': 'm.2', 'nvme': 'nvme', 'ssd nvme': 'nvme', 'sshd (solid state hybrid drive)': 'sshd', 'sshd': 'sshd', 'ssd (solid state drive)': 'ssd', 'ssd': 'ssd', 'hdd (hard disk drive)': 'hdd', 'hdd': 'hdd'}
df['storage type'] = df['storage type'].str.lower().apply(lambda x: next((v for k, v in storage_map.items() if k in x), x))

display(df)


## Storage Capacity TB to GB conversion, remove GB suffix

In [ ]:
# Convert TB to GB
df['storage capacity'] = df['storage capacity'].astype(str)
df['storage capacity'] = df['storage capacity'].str.replace('1tb', '1024gb')
df['storage capacity'] = df['storage capacity'].str.replace('2tb', '2048gb')
df['storage capacity'] = df['storage capacity'].str.replace('3tb', '3072gb')
df['storage capacity'] = df['storage capacity'].str.replace('4tb', '4096gb')
df['storage capacity'] = df['storage capacity'].str.replace('5tb', '5120gb')
df['storage capacity'] = df['storage capacity'].str.replace('6tb', '6144gb')
df['storage capacity'] = df['storage capacity'].str.replace('7tb', '7168gb')
df['storage capacity'] = df['storage capacity'].str.replace('8tb', '8192gb')
df['storage capacity'] = df['storage capacity'].str.replace('9tb', '9216gb')
df['storage capacity'] = df['storage capacity'].str.replace('10tb', '10240gb')
df['storage capacity'] = df['storage capacity'].str.replace('11tb', '11264gb')
df['storage capacity'] = df['storage capacity'].str.replace('12tb', '12288gb')
df['storage capacity'] = df['storage capacity'].str.replace('13tb', '13312gb')
df['storage capacity'] = df['storage capacity'].str.replace('14tb', '14336gb')
df['storage capacity'] = df['storage capacity'].str.replace('15tb', '15360gb')
df['storage capacity'] = df['storage capacity'].str.replace('16tb', '16384gb')


# Remove GB from Storage Capacity
df['storage capacity'] = df['storage capacity'].str.replace('gb', '')

# Drop rows with non numerical values in the 'storage capacity' column
df = df[pd.to_numeric(df['storage capacity'], errors='coerce').astype(float).notnull()]

# Drop rows where 'storage capacity' contains a decimal point
df = df[~df['storage capacity'].astype(str).str.contains('\.')]

# Drop rows where 'storage capacity' is less than 64
df = df[df['storage capacity'].astype(int) >= 64]


display(df)

## Remove GB suffix

In [ ]:
# Remove GB from RAM size
df['ram size'] = df['ram size'].astype(str)
df['ram size'] = df['ram size'].str.replace('gb', '')

# If ram size = storage capacity, set ram size to 8gb
df.loc[df['ram size'] == df['storage capacity'], 'ram size'] = '8'

# Drop rows with non-numerical values in the 'ram size' column
df = df[pd.to_numeric(df['ram size'], errors='coerce').astype(float).notnull()]

# Drop rows where ram size is more than 64gb
df = df[df['ram size'].astype(int) <= 64]

display(df)

# Convert screen size values

In [ ]:
# Strip all non-numerical characters from the 'screen size' column
import re

df['screen size'] = df['screen size'].apply(lambda x: re.findall('\d+\.\d+|\d+', str(x))[0])
display(df)


## GPU Column cleanup

In [ ]:
# Convert NaN entries in the 'gpu' column to 'integrated'
df['gpu'] = df['gpu'].fillna('integrated')

# Convert enties conatining 'intel' but no 'nvidia' or 'amd' to 'integrated'
df.loc[df['gpu'].str.contains('intel|integrated|hd|uhd|onboard') & ~df['gpu'].str.contains('nvidia|amd'), 'gpu'] = 'integrated'

# If entry contains '&' split string at '&' and keep the second part without the '&' symbol
df.loc[df['gpu'].str.contains('&|\+'), 'gpu'] = df['gpu'].str.split('&|\+').str[1]

# Drop 'graphics processing type' column
df.drop(columns=['graphics processing type'], inplace=True)

# If 'gpu' entry contains 'gtx' or 'radeon' but does not contain 'nvidia' or 'amd' respectively, add 'nvidia' or 'amd' to the entry
df.loc[df['gpu'].str.contains('gtx') & ~df['gpu'].str.contains('nvidia'), 'gpu'] = 'nvidia ' + df['gpu']
df.loc[df['gpu'].str.contains('radeon') & ~df['gpu'].str.contains('amd'), 'gpu'] = 'amd ' + df['gpu']

# Strip whitespace from 'gpu' column
df['gpu'] = df['gpu'].str.strip()

display(df)

## Remove entries with no sold date

In [ ]:
# Drop rows with no sold date
df.dropna(subset=['sold_date'], inplace=True)

# Save to CSV
df.to_csv('C:/Users/jack/Documents/temp.csv', index=False)

## Model/Series column clean up

In [ ]:
# Assuming you have a Pandas DataFrame called 'df' with a 'model' column
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


# Read the CSV file into a Pandas DataFrame
df2 = pd.read_csv('C:/Users/jack/Documents/temp.csv')

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv('C:/Users/jack/Documents/EbayMarketScraping/Scraping_bs4_ebay-main/Master Analytics - Aiken Import Dump.csv')

# Print the updated DataFrame
print(df)

# Designate Product Type to filter
filter = 'LAPTOP'

# Drop rows that do not contain 'LAPTOP' in the 'Product Type' column
df = df[df['ProductType'].str.contains(filter, na=False)]

# Convert the 'Model' column to a string
df['Model'] = df['Model'].astype(str)

# Create a list of unique entries in the 'Model' column to compare against
model_list = df['Model'].unique().tolist()

# Lowercase all strings in the list
model_list = [string.lower() for string in model_list]



# If 'model' entry string in df2 is only numeric, append 'series' entry to start of the string
for i, model in enumerate(df2['model']):
    if str(model).isnumeric():
        df2.at[i, 'model'] = str(df2.at[i, 'series']) + ' ' + model


# Loop through each entry in the 'model' column and replace it with the most similar entry from the model_list
for i, model in enumerate(df2['model']):
    # Use the process.extractOne() method to find the most similar entry in the model_list
    best_match = process.extractOne(str(model), model_list)
    # If the similarity score is above a certain threshold, replace the original entry with the best match
    if best_match[1] > 75:
        df2.at[i, 'model'] = best_match[0]
    if best_match[1] < 75:
        df2.drop(i, inplace=True)    

'''
# If 'model' entry string in df2 contains corresponding 'brand' string in df2, remove brand string from model string
for i, model in enumerate(df2['model']):
    if df2.at[i, 'brand'] in str(model):
        df2.at[i, 'model'] = model.strip(df2.at[i, 'brand'])
'''

# Strip 'notebook' and 'pc', 'laptop' and whitespace from the model entries
df2['model'] = df2['model'].str.replace('notebook', '')
df2['model'] = df2['model'].str.replace('pc', '')
df2['model'] = df2['model'].str.replace('laptop', '')
df2['model'] = df2['model'].str.strip()


# Drop rows where model entry string = 'nan'
df2.drop(df2[(df2['model'] == 'cheap gaming laptop') | (df2['model'] == 'nan')].index, inplace=True)   




display(df2)        

# Save dataframe as csv file

In [ ]:
# Save df as a CSV file and display it
df2.to_csv('C:/Users/jack/Documents/02-11-23_clean_output8719row.csv', encoding='utf-8', index=False)

df = pd.read_csv('C:/Users/jack/Documents/02-11-23_clean_output8719row.csv')
display(df)

In [ ]:
# Count and list all unique entries in the 'model' column
model_list = df['model'].unique().tolist()
print(model_list)
print(len(model_list))


In [ ]:
# Read the CSV file into a Pandas DataFrame
df_mod = pd.read_csv('E:\EbayMarketScraping\Scraping_bs4_ebay-main\Master Analytics - Aiken Import Dump.csv')

# Print the updated DataFrame
print(df_mod)

# Designate Product Type to filter
filter = 'LAPTOP'

# Drop rows that do not contain 'LAPTOP' in the 'Product Type' column
df_mod = df_mod[df_mod['ProductType'].str.contains(filter, na=False)]

# Convert the 'Model' column to a string
df_mod['Model'] = df_mod['Model'].astype(str)

# Create a list of unique entries in the 'Model' column
model_list = df_mod['Model'].unique().tolist()

# Add the 'Manufacturer' entry to the beginning of each string in the list
manufacturer_model_list = [df_mod.loc[df_mod['Model'] == model, 'Manufacturer'].fillna('').iloc[0] + ' ' + model for model in model_list if 'APPLE' not in model]

# Remove any string that contains the word 'APPLE'
query_list = [string for string in manufacturer_model_list if 'APPLE' not in string]

# Lowercase all strings in the list
query_list = [string.lower() for string in query_list]

# Remove 'notebook' and 'pc' and 'laptop' from the entries is the list
query_list = [string.replace('notebook', '') for string in query_list]
query_list = [string.replace('pc', '') for string in query_list]
query_list = [string.replace('laptop', '') for string in query_list]

# Strip whitespace from entries in the list
query_list = [string.strip() for string in query_list]

# Print the list of strings with the 'Manufacturer' entry added
print(manufacturer_model_list)
print(len(manufacturer_model_list))
print(query_list)
print(len(query_list))